# FDTD Sparameters in Meep

Here are some examples on how to extract Sparameters in Meep.

For extracting Sparameters, `gmeep` automatically moves the source between ports.

- add monitors
- run simulation
- pull coefficients, and create proper ratios to get Sparameters. Monitors are recording Fourier Transform fields. Sparameter is a relationship of those parameters. Frequency domain approach at many different frequencies. Get eigenmode coefficients.
    - forward coefficient: how much waveguide forward mode
    - backward coefficient: how much waveguide forward mode
   

Notice that most examples run with `resolution=20` so they run fast.

The resolution is in pixels/um so i reccommend that you run with at least `resolution=100` for 1/100 um/pixel (10 nm/ pixel)

   
## Single core

To run the examples you can use low `resolution=20` to speed up the simulations.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm

gf.config.set_plot_options(show_subports=False, show_ports=False)

In [ ]:
c = gf.components.straight(length=2)
p = 2
c = gf.add_padding_container(c, default=0, top=p, bottom=p)
c

`run=False` only plots the simulations for you to review that is set up **correctly**

In [ ]:
df = gm.write_sparameters_meep(c, run=False)

In [ ]:
df = gm.write_sparameters_meep(c, resolution=20)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

For a straight waveguide S21 (Transmission) is around 0dB (100% transmission)

## Port symmetries

You can save some simulations in reciprocal devices. 

In [ ]:
c = gf.components.bend_euler(radius=3)
p = 2
c = gf.add_padding_container(c, default=0, bottom=p, right=p)
c

In [ ]:
df = gm.write_sparameters_meep(c, run=True, port_symmetries=gm.port_symmetries.port_symmetries_1x1, resolution=20)
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s11m',))

In [ ]:
c = gf.components.crossing()
c

In [ ]:
port_symmetries = {
    "o1": {
        "s11": ["s22", "s33", "s44"],
        "s21": ["s12", "s34", "s43"],
        "s31": ["s13", "s24", "s42"],
        "s41": ["s14", "s23", "s32"],
    }
}
df = gm.write_sparameters_meep(c, resolution=20, port_symmetries=gm.port_symmetries.port_symmetries_crossing)

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=('s31m',))

# As you can see this crossing looks beautiful but it's not optimized

## Multicore

You can divide each simulation into multiple cores thanks to MPI.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import gdsfactory as gf
import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
component_name = 'mmi1x2'
c = gf.components.factory[component_name]()
p = 3
c = gf.add_padding_container(c, default=0, top=p, bottom=p)
c

In [ ]:
filepath = gm.write_sparameters_meep_mpi(
    component=c,
    cores=3,
    resolution=20
)

In [ ]:
filepath

In [ ]:
while not filepath.exists():
    time.sleep(1)
    print('waiting for sim result in {filepath!r}')

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=['s12m', 's13m'])

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=['s12m'])

## Multicore pool

You can also run a pool of multicore simulations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdsfactory as gf

import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
c = gf.components.straight(length=3)
p = 2
c = gf.add_padding_container(c, default=0, top=p, bottom=p)
c

In [ ]:
c1_dict = {
        "component": c,
    }
jobs = [
    c1_dict,
]

filepaths = gm.write_sparameters_meep_mpi_pool(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    lazy_parallelism=True,
    port_symmetries=gm.port_symmetries.port_symmetries_1x1
)

In [ ]:
df = pd.read_csv(filepaths[0])
gf.simulation.plot.plot_sparameters(df)

In [ ]:
component_name = 'coupler_ring'
c = gf.components.factory[component_name]()
p = 3
c = gf.add_padding_container(c, default=0, left=p, right=p, bottom=p)
c

In [ ]:
c1_dict = {
        "component": c,
    }
jobs = [c1_dict]

filepaths = gm.write_sparameters_meep_mpi_pool(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    delete_temp_files=False,
    lazy_parallelism=True,
)

In [ ]:
df = pd.read_csv(filepaths[0])

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=['s31m'])

In [ ]:
gm.plot.plot_sparameters(df, keys=['s41m'])